# Aws Athena

### Introduction

In this lesson, we'll see how we can work with AWS athena.  Aws athena is a service that allows us to query our data directly from an s3 bucket.  Let's get started.

### Benefits of Athena

Athena will allow us to query our S3 data without setting up or running a database like our RDS postgres image.

This has some benefits.  Remember that with a postgres image in RDS, we have to be careful about keeping this database running, especially if we are not using it very often.  Essentially, we'll be paying for idle time.  With athena, by contrast, our data is stored in a file in S3.  And nothing is running until we call the query.

Another benefit of athena is that we do not have to set up a traditional schema.  So we can query unstructured data with athena.

Still, athena has it's downsides.  It doesn't support typical database features like indexing, which is used to speed up our queries.  And with athena, we pay per query, so if running a lot of queries can have the cost add up.  

For these reasons, Athena is typically used for a couple of adhoc queries, before moving the data to a database.  Or it's used for some initial searching through unstructured data like log files.  Ok, let's start using it.

### Using athena with S3 

As we know Athena will allow us to query data directly from S3, and this data should be in Json or CSV form.  

If we are storing JSON data, the data needs to be in a specific form that looks like the following.

```python
{"song": "royals", "artist": "lorde"}
{"song": "taxman", "artist": "the beatles"}
{"song": "paint it black", "artist": "rolling stones"}
```

So with Json, each dictionary should be on a separate line in our s3 buckets, there should be no comma between our dictionaries, and we should not have any square brackets at the beginning or end of our list of dictionaries.

For CSV data, we can just upload a standard csv file to s3 (which is what we'll do).

### Storing our data

If you look at the `src/console.py` and `index.py` files, you can see how we accomplish this.  Looking at the `console.py` file:

* We create a new bucket to store our data to query (you'll have to set a unique name).
* We retrieve a list of dictionaries with a call to `find_receipts`. 
* We then use `pd.DataFrame` to convert this list of dictionaries to a dataframe.
* Then if you uncomment the `s3.upload_file` method, you'll see that we add this csv file to our bucket.

> You'll have to specify a bucket name.

We can confirm that this works, by then reading from the bucket like so.

```python
bucket_name = ''
object_name = ''
obj = s3.get_object(Bucket=bucket, Key=object_name)
text = obj['Body'].read()
```

Ok, so we've just created a bucket, and uploaded some CSV data for Athena to read our data from.

The next step is to create a bucket to *write to*.  It turns out that athena will be writing the results of our query to an object in a bucket, so let's create this bucket to store the output of a query.  

You can see in the `console.py` file, that we have a couple of lines for creating just this bucket.

```python
bucket_name = 'jigsawtexasresults' # replace bucket name
results_bucket = s3.create_bucket(Bucket = bucket_name)
```

Ok, so go to the s3 console, and confirm that our buckets have been created, and the query file has been uploaded.  

### Setting up Athena

Ok, now that we set up our S3 buckets, it's time to work with Athena itself.  So type athena in the search console, and then click on athena.

<img src="./athena.png" width="100%">

From there, click on Query your data, and click Launch query editor.

<img src="./query-data.png">

When we get into Athena, we'll see in the light blue banner at the top that `Before you run your first query, you need to set up a query result location in Amazon S3`.

<img src="./query-result-location.png" width="100%">

Let's do that now.  This is just the path to the results bucket that we created.

<img src="./results-bucket.png" width="80%">

So now that we have specified where Athena will place the results, the next step is to specify where we are getting our data from.  To do so, to the left, click on `Create`, and then AWS Glue Crawler as the source of the data that we will get our data from.

<img src="./aws-glue-crawler.png" width="50%">

By selecting AWS glue, we are instructing AWS to crawl the data in the specified s3 object, and then create a corresponding table from the attributes of our csv file.  

So let's do that.

### Creating our Glue Crawler

Go to the new page that popped up when clicking on AWS Glue Crawler, and follow the instructions of entering the crawler name, and then adding the data source.

> You can see that for this step of adding a data source, we can specify the **bucket** where we uploaded our data to.  Notice that we placed a `/` at the end of the bucket name, indicating to crawl the objects inside of the bucket.

<img src="./s3-bucket.png" width="70%">

After adding our S3 data source we should see something like the following.

<img src="./selected-source.png" width="100%">

Next we will need a new iam role to read from our s3 bucket.  Click on `Create new IAM` role.  And you can view the default IAM configuration created for you by clicking on `View`.

<img src="./iam-role.png" width="70%">

There, if you expand the S3 policy, this should make some sense.

<img src="./s3-access.png" width="100%">

We can see that for our query bucket, we are granting Glue GetObject and PutObject access on our specified query bucket.

Ok, next is to choose a Target database.

<img src="./create_db.png" width="80%">

Click on add a database, and fill in a database name.  From there, if you click on the refresh button to the right, you will be able to see your database, and select it.

<img src="./select-new-db.png" width="80%">

> This database is created in something called the AWS Lake Formation.

We can keep the database schedule as on demand.  Glue has the ability to recrawl our buckets on a schedule in case the structure of our data changes.

> Keep the schedule as `on demand`.

Finally, we can select create crawler.

<img src="./create-crawler.png" width="70%">

If it worked, you should see a green banner saying that the crawler was created, and from there you can click on `Run crawler` to the right.

<img src="./run-crawler.png" width="100%">

> If you see an error, one option is to type in lake formation, and make sure that there is admin access (when prompted).  Another way to troubleshoot is to look through the list of IAM users and make sure that none of the access keys are compromised.  If they are, you should reset them or delete the account.

### Back to Athena

Ok, so remember that Glue just turned crawled our S3 bucket so that we could query this bucket as a table.  Now we can go back to Athena to perform some queries.

> For the database, select the database that we created in athena.  And then we can query our bucket as if it were a table.  

So below, our query is:

```sql
select location_name, liquor_receipts from jigsawtexasquery where liquor_receipts = 0 limit 3;
```

<img src="./query-athena.png" width="100%">

Finally, like everything, it is also possible to access use Athena from boto3.  You can take a look at that in the `src/athena_boto.py` file, and we can talk through it in the next lesson.

### Resources

[AWS glue](https://docs.aws.amazon.com/glue/latest/dg/components-key-concepts.html)

[AWS Athena](https://www.sqlshack.com/an-introduction-to-aws-athena/)

[Athena pros and cons](https://towardsaws.com/aws-athena-why-is-it-different-than-mysql-93d55fd4a757)